In [25]:
global new_info
new_info=[]

global used_combos
used_combo=[]


class DataProcessing():
    
    def __init__(self,path,combos):
        self.path=path
        self.combos=combos
        
    def read_data(self):
        df=pd.read_csv(self.path)
        combo_list=df[self.combos].tolist()
        return df,combo_list
    
    def create_input_list(self,new_info):
        df=self.read_data()[0]
        cols=df.columns.tolist()
        cols.remove('Complementary')
        new_inputs=[]
        for i in new_info:
            line=df.iloc[i[0]]
            vals=[line[j] for j in cols]
            vals.append(i[1])
            new_inputs.append(vals)
        input_df=pd.DataFrame(data=new_inputs,columns=df.columns)
        return input_df
    
    def extract_info(self,com):
        line=df.iloc[com]
        article1=line['ID']
        article2=line['ID.1']
        string=str(article1) +' '+ str(article2)
        return string
    
    

In [26]:
import tkinter as tk
import pandas as pd
import numpy as np


class ButtonPresses(DataProcessing):
    
    root=tk.Tk()
    
    def __init__(self,path,combos):
        DataProcessing.__init__(self,path,combos)
        self.root=tk.Tk()
        self.df=pd.read_csv(path)
        self.df=self.read_data()[0]
        self.combo_list=self.read_data()[1]
        
    def press1(self):
        r=np.random.randint(1,len(combo_list))
        new_info.append([r,1])
        print(new_info)
        pair=self.extract_info(r)
        print(pair)
        #self.set(pair)
        self.label_text=pair
        self.topframe.config(text=pair)
        used_combo.append(r)
        combo_list.remove(r)
    
    
    def press2(self):
        r=np.random.randint(1,len(combo_list))
        new_info.append([r,2])
        print(new_info)
        pair=self.extract_info(r)
        print(pair)
        #self.set(pair)
        self.label_text=pair
        self.topframe.config(text=pair)
        used_combo.append(r)
        combo_list.remove(r)

    def press3(self):
        r=np.random.randint(1,len(combo_list))
        new_info.append([r,3])
        print(new_info)
        pair=self.extract_info(r)
        print(pair)
        #self.set(pair)
        self.label_text=pair
        self.topframe.config(text=pair)
        used_combo.append(r)
        combo_list.remove(r)
    
    def erase_inputs(self):
        
        global new_info
        new_info=[]
        
        combo_list.append(used_combo)
        
        global used_combo
        used_combo=[]

        

SyntaxError: name 'used_combo' is used prior to global declaration (<ipython-input-26-b9e069f31b4e>, line 64)

In [27]:
import tkinter as tk
import pandas as pd
import numpy as np
from tkinter import *
from tkinter.ttk import * 
global root
root=tk.Tk()
class FullGUI(ButtonPresses):
    
    root=tk.Tk()
    label_text='combo'
    def __init__(self,path,combos):
        
        ButtonPresses.__init__(self,path,combos)
        self.path=path
        self.combos=combos
        self.root=tk.Tk()
        self.topframe=tk.Frame(self.root)
        self.topframe=tk.Label(text=self.label_text,width=20,height=20,bg='floral white')
        self.topframe.pack(side=tk.TOP)
        
        
        self.leftframe=tk.Frame(self.root)
        self.leftframe=tk.Button(text="Dislike",command=self.press1,bg='light blue')
        self.leftframe.pack(side=tk.LEFT)
        
        self.rightframe=tk.Frame(self.root)
        self.rightframe=tk.Button(text="Love",command=self.press3,bg='pink')
        self.rightframe.pack(side=tk.RIGHT)
        
        
        self.bottom2frame=tk.Frame(self.root)
        self.bottom2frame=tk.Button(text="Erase",command=self.erase_inputs,bg='plum1')
        self.bottom2frame.pack(side=tk.BOTTOM)
        
        self.bottomframe=tk.Frame(self.root,bg='white')
        self.bottomframe=tk.Button(text="Neutral",command=self.press2, bg='light yellow')
        self.bottomframe.pack(side=tk.BOTTOM)
        
        
        
        
        
        #self.root.configure(background='old lace')
        
        root.mainloop()

In [28]:
from sklearn.svm import SVC

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import LabelEncoder

import xgboost as xgb

from sklearn.model_selection import train_test_split


class MachineLearning():
    
    def __init__(self, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs
        self.var=inputs[outputs]
        self.listed_insts=[SVC(),LogisticRegression(),RandomForestClassifier(),DecisionTreeClassifier(),xgb.XGBClassifier()]
    
    def handle_strings_in(self):
        df=self.inputs
        objects=[i for i in df.columns if type(self.inputs[i][1]) is str]
        numerical=[i for i in self.inputs.columns if type(self.inputs[i][1]) is np.float64]
        
        dummy_df=pd.get_dummies(self.inputs[objects])
        self.inputs.drop(objects,axis=1,inplace=True)
        
        complete_input=pd.concat([self.inputs,dummy_df],axis=1)
        
        return complete_input
    
    
    def handle_strings_out(self):
        
        encoder_inst=LabelEncoder()
        complete_output=encoder_inst.fit_transform(self.var)
        
        return complete_output
    
    
    def evaluate_models(self):
        inst_list=self.listed_insts
        global X
        X=self.handle_strings_in()
        global y
        y=self.handle_strings_out()
        score_mat=[]
        for i in inst_list:
            scores=cross_val_score(i,X,y,cv=4)
            score_mat.append(scores)
            
        return score_mat
    
    def predict_with_best_model(self):
        
        model_score=self.evaluate_models()
        
        avg_scores=[np.mean(i) for i in model_score]
        
        best=avg_scores.index(np.max(avg_scores))
    
        list_inst=self.listed_insts
        inst=list_inst[best]
        
        X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2)
        fit_inst=inst.fit(X_train,y_train)
        preds=fit_inst.predict(X_test)
        acc=accuracy_score(preds,y_test)
        return preds,acc
        
        
        
        
        

In [29]:
dp=DataProcessing('../Data/combo_df','combo_index')

df=dp.read_data()[0]
combo_list=dp.read_data()[1]

In [30]:
bp=ButtonPresses('../Data/combo_df','combo_index')


In [31]:

mg=FullGUI('../Data/combo_df','combo_index')

[[1393, 1]]
28 44
[[1393, 1], [1575, 2]]
32 26
[[1393, 1], [1575, 2], [490, 2]]
10 41
[[725, 1]]
15 26
[[725, 1], [2483, 1]]
50 34
[[725, 1], [2483, 1], [635, 2]]
13 36


In [9]:
input_df=mg.create_input_list(new_info)
ml=MachineLearning(input_df,'Complementary')
#ml.evaluate_models()
ml.predict_with_best_model()

C:\Users\603766\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\603766\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\603766\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\U

(array([0, 0, 2, 1, 2, 0], dtype=int64), 1.0)